In [349]:
import numpy as np
import pandas as pd
from numba import njit


# 遷移確率のCSVファイルを読み込む
change_probability = pd.read_csv('senni_pro.csv')
variable = pd.read_csv('Variable.csv')
markov_state = pd.read_csv('markov_state.csv')
test_quality = pd.read_csv('test_quality.csv')
symptom = pd.read_csv('symptom.csv')
cost = pd.read_csv('cost.csv')
utility = pd.read_csv('utility.csv')
setting = pd.read_csv('setting.csv')
accident_rate = pd.read_csv('accident_rate.csv')

# Model input
n_i = 10000  # number of simulated individuals
n_t = 44  # time horizon, 30 cycles
# markov_stateの2列目の値を取得し、1行目を除いてリスト化
v_n = markov_state.iloc[0:, 1].tolist()
n_s = len(v_n)  # the number of health states
v_M_1 = ["H"] * n_i  # everyone begins in the healthy state
d_c = d_e = 0.02  # equal discounting of costs and QALYs by 3%
v_Trt = ["No Treatment", "Treatment"]  # store the strategy names
print(v_n)
print(len(v_n))

['H', 'LR1', 'LR2', 'HR', 'DA', 'DAU', 'DB', 'DBU', 'DC', 'DCU', 'DD', 'DDU', 'D', 'PRH', 'PDAH', 'PDBH', 'PDCH', 'PDDH']
18


In [350]:
def Probs(M_it, stage,dur):
    """指定されたステージのマルコフ状態に対する遷移確率を計算する."""
    # ステージごとの遷移確率を取得
    p_normal_to_lr1 = change_probability.iloc[stage, 1]
    p_lr1_to_lr2 = change_probability.iloc[stage, 2]
    p_lr2_to_hr = change_probability.iloc[stage, 3]
    p_hr_to_da = change_probability.iloc[stage, 4]
    p_da_to_db = change_probability.iloc[stage, 5]
    p_da_to_death = change_probability.iloc[stage, 6]
    p_db_to_dc = change_probability.iloc[stage,7]
    p_db_to_death = change_probability.iloc[stage, 8]
    p_dc_to_dd = change_probability.iloc[stage, 9]
    p_dc_to_death = change_probability.iloc[stage, 10]
    p_dd_to_death = change_probability.iloc[stage, 11]
    p_prn_to_lr1 = change_probability.iloc[stage, 12]
    p_prn_to_lr2 = change_probability.iloc[stage, 13]
    p_prn_to_hr = change_probability.iloc[stage, 14]
    p_r_dukeA = change_probability.iloc[stage, 15]
    p_r_dukeB = change_probability.iloc[stage, 16]
    p_r_dukeC = change_probability.iloc[stage, 17]	
    p_r_dukeD = change_probability.iloc[stage, 18]
    p_DA_sy   = 0.065 #DukeAで症状がある割合
    p_DB_sy   = 0.26
    p_DC_sy   = 0.46
    p_DD_sy   = 0.92

    #DAU, DBU, DCU, DDUに関してはマイクロシミュレーション的に分類させる必要がある。
    v_p_it = np.zeros(n_s)
    if M_it == "H":
        v_p_it[:] = [1 - p_normal_to_lr1, p_normal_to_lr1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    elif M_it == "LR1":
        v_p_it[:] = [0, 1 - p_lr1_to_lr2, p_lr1_to_lr2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    elif M_it == "LR2":
        v_p_it[:] = [0, 0, 1 - p_lr2_to_hr, p_lr2_to_hr, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    elif M_it == "HR":
        v_p_it[:] = [0, 0, 0, 1 - p_hr_to_da, p_hr_to_da, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

    elif M_it == "DA":
        v_p_it[:] = [0, 0, 0, 0, (1-p_DA_sy)*(1-p_da_to_db-p_da_to_death),p_DA_sy*(1-p_da_to_death),(1-p_DA_sy)*p_da_to_db, 0, 0, 0, 0, 0, p_da_to_death, 0, 0, 0, 0, 0]
    elif M_it == "DAU":
        if dur >= 4:  # 4回続いた場合
            v_p_it[:] = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
        else:
            v_p_it[:] = [0, 0, 0, 0, 0, 1-p_da_to_death, 0, 0, 0, 0, 0, 0, p_da_to_death, 0, 0, 0, 0, 0]
    elif M_it == "DBU":
        if dur >= 4:  # 4回続いた場合
            v_p_it[:] = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
        else:
            v_p_it[:] = [0, 0, 0, 0, 0, 0, 0, 1 - p_db_to_death, 0, 0, 0, 0, p_db_to_death, 0, 0, 0, 0, 0]
    elif M_it == "DCU":
        if dur >= 4:  # 4回続いた場合
            v_p_it[:] = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
        else:
            v_p_it[:] = [0, 0, 0, 0, 0, 0, 0, 0, 0, 1 - p_dc_to_death, 0, 0, p_dc_to_death, 0, 0, 0, 0, 0]

    elif M_it == "DDU":
        if dur >= 4:  # 4回続いた場合
            v_p_it[:] = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
        else:
            v_p_it[:] = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1 - p_dd_to_death, p_dd_to_death, 0, 0, 0, 0, 0]
    elif M_it == "DB":
        v_p_it[:] = [0, 0, 0, 0, 0, 0, (1-p_DB_sy)*(1-p_db_to_dc-p_db_to_death), p_DB_sy*(1-p_db_to_death), (1-p_DB_sy)*p_db_to_dc, 0, 0, 0, p_db_to_death, 0, 0, 0, 0, 0]
    elif M_it == "DC":
        v_p_it[:] = [0, 0, 0, 0, 0, 0, 0, 0, (1 - p_DC_sy)*(1 - p_dc_to_dd- p_dc_to_death), p_DC_sy*(1-p_dc_to_death), (1 - p_DC_sy)*p_dc_to_dd, 0, p_dc_to_death, 0, 0, 0, 0, 0]
    elif M_it == "DD":
        v_p_it[:] = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, (1 - p_DD_sy)*(1 - p_dd_to_death), p_DD_sy*(1 - p_dd_to_death), p_dd_to_death, 0, 0, 0, 0, 0]
    elif M_it == "D":
        v_p_it[:] = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]

    elif M_it == "PRH":
        v_p_it[:] = [0, p_prn_to_lr1, p_prn_to_lr2, p_prn_to_hr,0 , 0, 0, 0, 0, 0, 0, 0, 0, 1 - p_prn_to_lr1 - p_prn_to_lr2 - p_prn_to_hr, 0, 0, 0, 0]
    elif M_it == "PDAH":
        v_p_it[:] = [0, 0, 0, 0, p_r_dukeA, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1-p_r_dukeA, 0, 0, 0]
    elif M_it == "PDBH":
        v_p_it[:] = [0, 0, 0, 0, 0, 0, p_r_dukeB, 0, 0, 0, 0, 0, 0, 0, 0, 1-p_r_dukeB, 0, 0]
    elif M_it == "PDCH":
        v_p_it[:] = [0, 0, 0, 0, 0, 0, 0, 0, p_r_dukeC, 0, 0, 0, 0, 0, 0, 0, 1-p_r_dukeC, 0]
    elif M_it == "PDDH":
        v_p_it[:] = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, p_r_dukeD, 0, 0, 0, 0, 0, 0, 1-p_r_dukeD]
   # 出力の確認
    assert np.isclose(v_p_it.sum(), 1, atol=1e-6), "Probabilities do not sum to 1"
    return v_p_it

In [351]:
# #遷移確率が全て正しくなっていることを確認した
# # 全ての状態のリストを定義
# all_states = ['H', 'LR1', 'LR2', 'HR', 'DA', 'DAU', 'DB', 'DBU', 'DC', 'DCU', 'DD', 'DDU', 'D', 'PRH', 'PDAH', 'PDBH', 'PDCH', 'PDDH']

# # ステージの数（例えば、change_probabilityデータフレームの行数）
# num_stages = len(change_probability)

# # 各状態とステージに対してProbs関数を呼び出し、遷移確率の合計が1になるかを確認
# for M_it in all_states:
#     for stage in range(num_stages):
#         output_probabilities = Probs(M_it, stage)
#         total_probability = sum(output_probabilities)
#         if not np.isclose(total_probability, 1, atol=1e-6):
#             print(f"Stage: {stage}, State: {M_it}, Total Probability: {total_probability}")
##ここから↓は独立している
# states = ["H", "LR1", "LR2", "HR", "DA", "DAU", "DB", "DBU", "DC", "DCU", "DD", "DDU", "D", "PRH", "PDAH", "PDBH", "PDCH", "PDDH"]
# stages = range(change_probability.shape[0])  # 変更確率のステージ数に応じて設定

# for stage in stages:
#     for state in states:
#         v_p_it = Probs(state, stage)
#         print(f"Stage {stage}, State {state}: {v_p_it}")
# print(type(v_p_it))


# change_probabilityをNumPy配列に変換
change_probability_array = change_probability.to_numpy()

# 結果の確認
for stage in stages:
    for state in states:
        state_index = state_dict[state]
        v_p_it = Probs_fast(state_index, stage, change_probability_array)
        print(f"Stage {stage}, State {state}: {v_p_it}")
print(type(v_p_it))

Stage 0, State H: [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Stage 0, State LR1: [0.    0.912 0.088 0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.   ]
Stage 0, State LR2: [0.    0.    0.923 0.077 0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.   ]
Stage 0, State HR: [0.    0.    0.    0.923 0.077 0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.   ]
Stage 0, State DA: [0.    0.    0.    0.    0.383 0.034 0.    0.    0.    0.    0.    0.
 0.583 0.    0.    0.    0.    0.    0.   ]
Stage 0, State DAU: [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Stage 0, State DB: [0.    0.    0.    0.    0.    0.    0.327 0.017 0.    0.    0.    0.
 0.656 0.    0.    0.    0.    0.    0.   ]
Stage 0, State DBU: [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Stage 0, State DC: [0.    0.    0.    0.    0.    0.    0.    0.    0.103 0.032 0.    0.
 0.865 0.

In [352]:
# コストデータの設定　numbaを使って高速化
# c_fit = cost.loc[0, 'value']
# c_ma_f1_dukesA = cost.loc[1, 'value']
# c_ma_f1_dukesB = cost.loc[2, 'value']
# c_ma_f1_dukesC = cost.loc[3, 'value']
# c_ma_f1_dukesD = cost.loc[4, 'value']
# c_ma_f2_dukesA = cost.loc[5, 'value']
# c_ma_f2_dukesB = cost.loc[6, 'value']
# c_ma_f2_dukesC = cost.loc[7, 'value']
# c_ma_f2_dukesD = cost.loc[8, 'value']
# c_rem_hr_ply = cost.loc[9, 'value']
# c_rem_lr_poly = cost.loc[10, 'value']
# c_tcs = cost.loc[11, 'value']
c_fit = 1557
c_ma_f1_dukesA = 1786482
c_ma_f1_dukesB = 2056922
c_ma_f1_dukesC = 2637803
c_ma_f1_dukesD = 3179764
c_ma_f2_dukesA = 34610
c_ma_f2_dukesB = 34610
c_ma_f2_dukesC = 43758
c_ma_f2_dukesD = 2476258
c_rem_hr_ply = 452445
c_rem_lr_poly = 48650
c_tcs = 30943

def Costs(M_it, next_M_it, p_it, Trt=False):
    """各マルコフ状態に対応するコストを計算する関数"""
    c_it = np.zeros_like(M_it, dtype=float)
    c_it[M_it == "H"] = 0
    c_it[M_it == "LR1"] = 0
    c_it[M_it == "LR2"] = 0
    c_it[M_it == "HR"] = 0
    # c_it[M_it == "DA"] = c_ma_f1_dukesA
    c_it[M_it == "DAU"] = c_ma_f2_dukesA
    # c_it[M_it == "DB"] = c_ma_f1_dukesB
    c_it[M_it == "DBU"] = c_ma_f2_dukesB
    # c_it[M_it == "DC"] = c_ma_f1_dukesC
    c_it[M_it == "DCU"] = c_ma_f2_dukesC
    # c_it[M_it == "DD"] = c_ma_f1_dukesD
    c_it[M_it == "DDU"] = c_ma_f2_dukesD
    c_it[M_it == "D"] = 0  # 死亡状態にはコストなし
    c_it[M_it == "PRH"] = 0  # 仮にコストなしと設定
    c_it[M_it == "PDAH"] = 0  # 仮にコストなしと設定
    c_it[M_it == "PDBH"] = 0  # 仮にコストなしと設定
    c_it[M_it == "PDCH"] = 0  # 仮にコストなしと設定
    c_it[M_it == "PDDH"] = 0  # 仮にコストなしと設定
    # DA -> DAU または症状ありで死亡する場合のコスト
    mask_DA_to_DAU = (M_it == "DA") & (next_M_it == "DAU")
    if np.any(mask_DA_to_DAU):
        c_it[mask_DA_to_DAU] = c_ma_f1_dukesA
    
    mask_DA_to_D = (M_it == "DA") & (next_M_it == "D") & (p_it[:, 5] > 0)
    if np.any(mask_DA_to_D):
        c_it[mask_DA_to_D] = c_ma_f1_dukesA * p_it[mask_DA_to_D, 5]
    
    # DB -> DBU または症状ありで死亡する場合のコスト
    mask_DB_to_DBU = (M_it == "DB") & (next_M_it == "DBU")
    if np.any(mask_DB_to_DBU):
        c_it[mask_DB_to_DBU] = c_ma_f1_dukesB
    
    mask_DB_to_D = (M_it == "DB") & (next_M_it == "D") & (p_it[:, 7] > 0)
    if np.any(mask_DB_to_D):
        c_it[mask_DB_to_D] = c_ma_f1_dukesB * p_it[mask_DB_to_D, 7]
    
    # DC -> DCU または症状ありで死亡する場合のコスト
    mask_DC_to_DCU = (M_it == "DC") & (next_M_it == "DCU")
    if np.any(mask_DC_to_DCU):
        c_it[mask_DC_to_DCU] = c_ma_f1_dukesC
    
    mask_DC_to_D = (M_it == "DC") & (next_M_it == "D") & (p_it[:, 9] > 0)
    if np.any(mask_DC_to_D):
        c_it[mask_DC_to_D] = c_ma_f1_dukesC * p_it[mask_DC_to_D, 9]
    
    # DD -> DDU または症状ありで死亡する場合のコスト
    mask_DD_to_DDU = (M_it == "DD") & (next_M_it == "DDU")
    if np.any(mask_DD_to_DDU):
        c_it[mask_DD_to_DDU] = c_ma_f1_dukesD
    
    mask_DD_to_D = (M_it == "DD") & (next_M_it == "D") & (p_it[:, 11] > 0)
    if np.any(mask_DD_to_D):
        c_it[mask_DD_to_D] = c_ma_f1_dukesD * p_it[mask_DD_to_D, 11]
    return c_it



# M_it=['H', 'LR1', 'LR2', 'HR', 'DA', 'DAU', 'DB', 'DBU', 'DC', 'DCU', 'DD', 'DDU', 'D', 'PRH', 'PDAH', 'PDBH', 'PDCH', 'PDDH']

# # コストを計算
# c_it = Costs(M_it)
# print(c_it)
# # 各状態のコストを表示
# print("H 状態のコスト:", c_it[M_it == "H"])
# print("LR1 状態のコスト:", c_it[M_it == "LR1"])
# print("LR2 状態のコスト:", c_it[M_it == "LR2"])
# print("HR 状態のコスト:", c_it[M_it == "HR"])
# print("DA 状態のコスト:", c_it[M_it == "DA"])
# print("DAU 状態のコスト:", c_it[M_it == "DAU"])
# print("DB 状態のコスト:", c_it[M_it == "DB"])
# print("DBU 状態のコスト:", c_it[M_it == "DBU"])
# print("DC 状態のコスト:", c_it[M_it == "DC"])
# print("DCU 状態のコスト:", c_it[M_it == "DCU"])
# print("DD 状態のコスト:", c_it[M_it == "DD"])
# print("DDU 状態のコスト:", c_it[M_it == "DDU"])
# print("D 状態のコスト:", c_it[M_it == "D"])
# print("PRH 状態のコスト:", c_it[M_it == "PRH"])
# print("PDAH 状態のコスト:", c_it[M_it == "PDAH"])
# print("PDBH 状態のコスト:", c_it[M_it == "PDBH"])
# print("PDCH 状態のコスト:", c_it[M_it == "PDCH"])
# print("PDDH 状態のコスト:", c_it[M_it == "PDDH"])

In [353]:
u_dukeA = utility.loc[0, 'value']
u_dukeB = utility.loc[1, 'value']
u_dukeC = utility.loc[2, 'value']
u_dukeD = utility.loc[3, 'value']
u_hl    = utility.loc[4, 'value']
u_lr1   = utility.loc[5, 'value']
u_lr2   = utility.loc[6, 'value']
u_no_sy = utility.loc[7, 'value']
u_normal= utility.loc[8, 'value']

def Effs(M_it, dur, Trt=False, cl=1, X=None):
    """各マルコフ状態に対応するユーティリティを計算する関数"""
    if X is None:
        X = np.ones_like(M_it, dtype=float)  # デフォルト値として1の配列を使用
    u_it = np.zeros_like(M_it, dtype=float)
    u_it[M_it == "H"] = u_normal
    u_it[M_it == "LR1"] = u_normal
    u_it[M_it == "LR2"] = u_normal
    u_it[M_it == "HR"] = u_normal
    u_it[M_it == "DA"] = u_dukeA
    u_it[M_it == "DAU"] = u_dukeA
    u_it[M_it == "DB"] = u_dukeB
    u_it[M_it == "DBU"] = u_dukeB
    u_it[M_it == "DC"] = u_dukeC
    u_it[M_it == "DCU"] = u_dukeC
    u_it[M_it == "DD"] = u_dukeD
    u_it[M_it == "DDU"] = u_dukeD
    u_it[M_it == "D"] = 0
    u_it[M_it == "PRH"] = u_normal
    u_it[M_it == "PDAH"] = u_normal
    u_it[M_it == "PDBH"] = u_normal
    u_it[M_it == "PDCH"] = u_normal
    u_it[M_it == "PDDH"] = u_normal
    return u_it * cl


In [354]:
print(change_probability)

    time  p_normal_to_lr1  p_lr1_to_lr2  p_lr2_to_hr  p_hr_to_da  p_da_to_db  \
0      0            0.088         0.077        0.077       0.034       0.583   
1      1            0.088         0.077        0.077       0.034       0.583   
2      2            0.088         0.077        0.077       0.034       0.583   
3      3            0.088         0.077        0.077       0.034       0.583   
4      4            0.088         0.077        0.077       0.034       0.583   
5      5            0.088         0.077        0.077       0.034       0.583   
6      6            0.088         0.077        0.077       0.034       0.583   
7      7            0.088         0.077        0.077       0.034       0.583   
8      8            0.088         0.077        0.077       0.034       0.583   
9      9            0.088         0.077        0.077       0.034       0.583   
10    10            0.088         0.077        0.077       0.034       0.583   
11    11            0.095         0.082 

In [355]:
def samplev(probs, m):
    n, k = probs.shape  # nは1、kはマルコフ状態の数、今回なら18が入る
    ran = np.empty((n, m), dtype=int)  # サンプリング結果を格納する配列
    U = np.cumsum(probs, axis=1)  # 横方向に累積和を計算
    if not np.allclose(U[:, -1], 1):
        raise ValueError("Probabilities do not sum to 1")

    for j in range(m):
        un = np.random.rand(n, 1)  # 各個人ごとにランダムな値を生成
        ran[:, j] = (un > U).sum(axis=1)  # サンプリングした値が累積確率を超える位置を求める、ranは1行18列
    return ran

In [356]:
def MicroSim(v_M_1, n_i, n_t, states, X=None, d_c=0.02, d_e=0.02, TR_out=True, TS_out=True, Trt=False, seed=1):
    np.random.seed(seed)
    v_dwc = 1 / (1 + d_c) ** np.arange(n_t + 1)  # コストの割引率
    v_dwe = 1 / (1 + d_e) ** np.arange(n_t + 1)  # QALYの割引率

    m_M = np.empty((n_i, n_t + 1), dtype='U4')
    m_C = np.zeros((n_i, n_t + 1))
    m_E = np.zeros((n_i, n_t + 1))

    m_M[:, 0] = v_M_1  # 初期状態を設定
    dur = np.zeros(n_i)  # 病気の期間を初期化
    m_C[:, 0] = Costs(m_M[:, 0], m_M[:, 0], np.zeros((n_i, n_s)), Trt)
    m_E[:, 0] = Effs(m_M[:, 0], dur, Trt, X=X)
    
    #人数分繰り返していく。
    for t in range(1, n_t + 1):
        m_p = np.array([Probs(state, t, dur[i]) for i, state in enumerate(m_M[:, t - 1])])  # 現在の状態から次の状態への遷移確率を計算
        next_states = np.array([states[i] for i in samplev(m_p, 1).flatten()]) #flattenとは、n次元のNumpy配列を1次元のNumpy配列に変換するメソッド
        m_C[:, t] = Costs(m_M[:, t - 1], next_states, m_p, Trt)
        m_M[:, t] = next_states
        m_E[:, t] = Effs(m_M[:, t], dur, Trt, X=X)
        dur = np.where(np.isin(m_M[:, t], ["DAU", "DBU", "DCU", "DDU"]), dur + 1, 0)
        if t % 10 == 0:
            print(f"\r{t / n_t * 100:.0f}% done", end="")

    tc = m_C @ v_dwc  # 割引後の総コスト
    te = m_E @ v_dwe  # 割引後の総QALYs
    tc_hat = np.mean(tc)  # 平均コスト
    te_hat = np.mean(te)  # 平均QALYs

    if TS_out:
        TS = pd.DataFrame(
            {f"Cycle_{i}": [f"{m_M[j, i]}->{m_M[j, i+1]}" for j in range(n_i)] for i in range(n_t)}
        )
    else:
        TS = None

    if TR_out:
        TR = pd.DataFrame(m_M).apply(pd.Series.value_counts).fillna(0).T / n_i
        TR = TR.reindex(columns=states, fill_value=0)
    else:
        TR = None

    results = {
        "m_M": m_M,
        "m_C": m_C,
        "m_E": m_E,
        "tc": tc,
        "te": te,
        "tc_hat": tc_hat,
        "te_hat": te_hat,
        "TS": TS,
        "TR": TR
    }
    return results

In [357]:
# シミュレーションの実行 (TR_out=True)
sim_no_trt_with_TR = MicroSim(v_M_1, n_i, n_t, v_n, X=None, d_c=d_c, d_e=d_e, Trt=False, TR_out=True, seed=100)

# 結果の表示
if sim_no_trt_with_TR["TR"] is not None:
    print("Transition probabilities (with TR_out=True):")
    print(sim_no_trt_with_TR["TR"])
else:
    print("TR not calculated.")

91% doneTransition probabilities (with TR_out=True):
         H     LR1     LR2      HR      DA     DAU      DB     DBU      DC  \
0   1.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000   
1   0.9095  0.0905  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000   
2   0.8298  0.1641  0.0061  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000   
3   0.7531  0.2275  0.0189  0.0005  0.0000  0.0000  0.0000  0.0000  0.0000   
4   0.6863  0.2774  0.0344  0.0019  0.0000  0.0000  0.0000  0.0000  0.0000   
5   0.6234  0.3195  0.0532  0.0039  0.0000  0.0000  0.0000  0.0000  0.0000   
6   0.5714  0.3449  0.0753  0.0083  0.0001  0.0000  0.0000  0.0000  0.0000   
7   0.5177  0.3713  0.0977  0.0129  0.0003  0.0000  0.0001  0.0000  0.0000   
8   0.4737  0.3899  0.1143  0.0214  0.0004  0.0000  0.0001  0.0000  0.0001   
9   0.4307  0.4029  0.1362  0.0286  0.0011  0.0000  0.0002  0.0000  0.0001   
10  0.3951  0.4076  0.1576  0.0369  0.0016  0.0001  0.0006  0.0001  0.0000   
11  0.3605 

In [358]:
# シミュレーションの実行
import time

start_time = time.time()

# No Treatment戦略のシミュレーション
sim_no_trt = MicroSim(v_M_1, n_i, n_t, v_n, X=None, d_c=d_c, d_e=d_e,TR_out=True, TS_out=True, Trt=False, seed=100)

# Treatment戦略のシミュレーション
sim_trt = MicroSim(v_M_1, n_i, n_t, v_n, X=None, d_c=d_c, d_e=d_e,TR_out=True, TS_out=True, Trt=True, seed=100)

comp_time = time.time() - start_time
print(f"Computation time: {comp_time:.2f} seconds")

# 平均コストと平均QALYsを表示
print("No Treatment strategy:")
print(f"Average cost: {sim_no_trt['tc_hat']}")
print(f"Average QALYs: {sim_no_trt['te_hat']}")

print("\nTreatment strategy:")
print(f"Average cost: {sim_trt['tc_hat']}")
print(f"Average QALYs: {sim_trt['te_hat']}")
print(sim_trt['tc'].sum())

91% doneComputation time: 713.14 seconds
No Treatment strategy:
Average cost: 606217.1356469209
Average QALYs: 29.276965091936674

Treatment strategy:
Average cost: 606217.1356469209
Average QALYs: 29.276965091936674
6062171356.469209


In [316]:
# Cost-effectiveness analysis
v_C = np.array([sim_no_trt["tc_hat"], sim_trt["tc_hat"]])
sd_C = np.array([np.std(sim_no_trt["tc"]), np.std(sim_trt["tc"])]) / np.sqrt(n_i)
v_E = np.array([sim_no_trt["te_hat"], sim_trt["te_hat"]])
sd_E = np.array([np.std(sim_no_trt["te"]), np.std(sim_trt["te"])]) / np.sqrt(n_i)

print(v_C)
print(sd_C)
print(v_E)
print(sd_E)
delta_C = v_C[1] - v_C[0]
delta_E = v_E[1] - v_E[0]

[1794438.02608155 1794438.02608155]
[92050.25638301 92050.25638301]
[29.16757396 29.16757396]
[0.07205287 0.07205287]


[1 2 3 0 5]
